# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [25]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [26]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,puerto ayora,-0.7393,-90.3518,24.19,92,100,2.24,EC,1691585649
1,1,aasiaat,68.7098,-52.8699,4.68,76,91,1.46,GL,1691585650
2,2,olonkinbyen,70.9221,-8.7187,5.93,97,100,7.84,SJ,1691585650
3,3,edinburgh of the seven seas,-37.0676,-12.3116,11.74,57,74,8.35,SH,1691585650
4,4,arkhangel'sk,64.5401,40.5433,26.26,38,2,2.89,RU,1691585650


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [27]:
%%capture --no-display

# Configure the map plot
full_city_map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    color = "City",
    geo = True,
    tiles = "OSM",
    size = "Humidity",
    frame_width = 800,
    frame_height = 600
)

# Display the map
full_city_map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [36]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_df = city_data_df[city_data_df["Max Temp"] > 10]
ideal_df = ideal_df[ideal_df["Max Temp"] < 22].dropna()

# Drop any rows with null values
ideal_df.dropna()

# Display sample data
ideal_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
3,3,edinburgh of the seven seas,-37.0676,-12.3116,11.74,57,74,8.35,SH,1691585650
5,5,bredasdorp,-34.5322,20.0403,18.99,61,0,4.83,ZA,1691585569
15,15,glendive,47.1053,-104.7125,14.79,94,0,0.00,US,1691585651
17,17,albany,42.6001,-73.9662,21.31,80,100,4.47,US,1691585398
18,18,adamstown,-25.0660,-130.1015,20.21,69,79,12.04,PN,1691585651


### Step 3: Create a new DataFrame called `hotel_df`.

In [38]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_df.copy(deep=True)
hotel_df = hotel_df[["City", "Country", "Lat", "Lng", "Humidity"]]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head(10)

,City,Country,Lat,Lng,Humidity,Hotel Name
3,edinburgh of the seven seas,SH,-37.0676,-12.3116,57,
5,bredasdorp,ZA,-34.5322,20.0403,61,
15,glendive,US,47.1053,-104.7125,94,
17,albany,US,42.6001,-73.9662,80,
18,adamstown,PN,-25.0660,-130.1015,69,
21,rozhyshche,UA,50.9154,25.2691,63,
22,tiksi,RU,71.6872,128.8694,52,
24,sinop,TR,41.7711,34.8709,62,
29,isafjordur,IS,66.0755,-23.1240,66,
32,chegdomyn,RU,51.1167,133.0167,71,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [39]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "apiKey": geoapify_key,
    "categories": "accommodation.hotel"
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    # YOUR CODE HERE
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{row['Lng']},{row['Lat']},{radius}"
    params["bias"] = f"proximity:{row['Lng']},{row['Lat']}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
edinburgh of the seven seas - nearest hotel: No hotel found
bredasdorp - nearest hotel: Victoria Hotel
glendive - nearest hotel: La Quinta Inn & Suites by Wyndham Glendive
albany - nearest hotel: No hotel found
adamstown - nearest hotel: No hotel found
rozhyshche - nearest hotel: Зелений гай
tiksi - nearest hotel: Арктика
sinop - nearest hotel: No hotel found
isafjordur - nearest hotel: Hótel Horn
chegdomyn - nearest hotel: Бурея
lompoc - nearest hotel: Embassy Suites by Hilton Lompoc Central Coast
selfoss - nearest hotel: Miðgarður
blackmans bay - nearest hotel: Villa Howden
badger - nearest hotel: Hotel North Pole
arraial do cabo - nearest hotel: No hotel found
valparaiso - nearest hotel: Ibis Hotel
preili - nearest hotel: No hotel found
hibbing - nearest hotel: Hibbing Park Hotel
dali - nearest hotel: 风花雪月大酒店
stanley - nearest hotel: Hotel 52
vallenar - nearest hotel: Hotel del Marqués
yellowknife - nearest hotel: No hotel found
mount gambier - nearest hotel: T

,City,Country,Lat,Lng,Humidity,Hotel Name
3,edinburgh of the seven seas,SH,-37.0676,-12.3116,57,No hotel found
5,bredasdorp,ZA,-34.5322,20.0403,61,Victoria Hotel
15,glendive,US,47.1053,-104.7125,94,La Quinta Inn & Suites by Wyndham Glendive
17,albany,US,42.6001,-73.9662,80,No hotel found
18,adamstown,PN,-25.0660,-130.1015,69,No hotel found
...,...,...,...,...,...,...
556,kirensk,RU,57.7853,108.1119,84,Ивушка
561,okhotsk,RU,59.3833,143.3000,79,No hotel found
569,byron bay,AU,-28.6500,153.6167,75,Lord Byron Resort
572,cangucu,BR,-31.3950,-52.6756,93,Hotel Brasil


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [43]:
%%capture --no-display

# Configure the map plot
hotel_map_plot = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    size = "Humidity",
    color = "City",
    frame_width=800,
    frame_height=600,
    hover_cols=["Lng", "Lat", "City", "Humidity", "Country", "Hotel Name"]
)

# Display the map
hotel_map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Hotel Name)